In [1]:
# Imports
import os
import csv
from datetime import datetime
from edc_constants.constants import NO, YES

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
data = []
with open('/Users/amediphoko/source/flourish/infant_feeding_dates_final.csv', 'r') as file:
    records = csv.DictReader(file)
    for row in records:
        data.append(row)

In [5]:
for record in data:
    update_data = {}
    if record.get('is_query') == NO:
        # Check match between date weaned or recent bf date, to update
        dt_weaned_match = record.get('dt_weaned_match')
        recent_bf_dt_match = record.get('recent_bf_dt_match')
        if dt_weaned_match == 'TRUE':
            dt_weaned = datetime.strptime(record.get('dt_weaned'), '%d/%m/%y').date()
            update_data['recent_bf_dt'] = dt_weaned
        elif recent_bf_dt_match == 'TRUE':
            recent_bf_dt = datetime.strptime(record.get('recent_bf_dt'), '%d/%m/%y').date()
            update_data['dt_weaned'] = recent_bf_dt
    elif record.get('is_query') == YES:
        confirmed_weaning = datetime.strptime(record.get('confirmed_weaning'), '%d/%m/%y').date()
        confirmed_solids = datetime.strptime(record.get('confirmed_solids'), '%d/%m/%y').date()
        update_data['dt_weaned'] = confirmed_weaning
        update_data['recent_bf_dt'] = confirmed_weaning
        update_data['solid_foods_dt'] = confirmed_solids

    # Update CRF data
    try:
        child_visit = ChildVisit.objects.get(
            subject_identifier=record.get('subject_identifier'),
            visit_code=record.get('visit_code'),
            visit_code_sequence=record.get('visit_code_sequence'))
    except ChildVisit.DoesNotExist:
        print(record.get('subject_identifier'), record.get('visit_code'), 'DNE')
    else:
        feeding_obj = InfantFeeding.objects.filter(child_visit=child_visit)
        if feeding_obj:
            feeding_obj.update(**update_data)
            continue
        print('No feeding instance', record.get('subject_identifier'), record.get('visit_code'))